# INSTALACIÓN DE LAS LIBRERÍAS

In [42]:
!rm -r /content/UNET5G

In [43]:
cd /content/

/content


In [44]:
!git clone https://github.com/IvanGarcia7/UNET5G.git

Cloning into 'UNET5G'...
remote: Enumerating objects: 4828, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 4828 (delta 9), reused 83 (delta 7), pack-reused 4743
Receiving objects: 100% (4828/4828), 206.91 MiB | 34.88 MiB/s, done.
Resolving deltas: 100% (1614/1614), done.
Updating files: 100% (13919/13919), done.


In [45]:
cd /content/UNET5G/Code/Pytorch-UNet

/content/UNET5G/Code/Pytorch-UNet


In [46]:
!pip install -r requirements.txt

In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
!pip3 install torch torchvision torchaudio

# PREPARACIÓN DE LOS DATOS


El modelo está diseñado para aceptar como entrada una matriz de tamaño fijo, por ejemplo 100 x 100 y con 8 dimensiones. Estas dimensiones estarían compuestas por:


*   Path_Loss
*   Respectivos SNR de cada estación base
*   Despliegue de cada una de las estaciones base
*   Despliegue de los usuarios


Como salida, el modelo genera una matriz con el mismo tamaño que una de las dimensiones de la matriz de entrada. Esta matríz tendrá una sola dimensión y contendrá información acerca del ancho de banda estimado.

A continuación, se especifíca el código necesario para generar una matriz de ejemplo, la cual se le dará como entrada al modelo.

In [47]:
!ls /content/UNET5G/SAMPLE/adaptive_0

0-0-4605876  0-0-4612497-0  0-0-4612497-1  0-0-4612497-2


## Generación de los datos en forma de matriz dado un directorio

In [53]:
!rm -r /content/Matrices

In [54]:
!mkdir /content/Matrices
!mkdir /content/Matrices/IN
!mkdir /content/Matrices/OUT
!mkdir /content/Matrices/TEST/
!mkdir /content/Matrices/TEST/IN
!mkdir /content/Matrices/TEST/OUT

In [55]:
import numpy as np
import os
from natsort import natsorted

#Ruta donde se encuentran los archivos, por ejemplo '/opt/share/MERIDA/Data/adaptive_0/0-0-4605876/ia/raw/'
folder = '/content/UNET5G/SAMPLE/adaptive_0/0-0-4605876/ia/raw/'

#Ruta donde se van a almacenar las matrices ya procesadas, por ejemplo '/opt/share/MERIDA/DATASET-TEST/'
folder_output = '/content/Matrices/'

#Indicar si se va a realizar o no estandarización de los datos
estandarizacion = False

lista = os.listdir(folder+'deployment/')
lista = natsorted(lista)

for i in range(0,len(lista)-3):
    deploy_femto = folder+'deployment/'+str(i)+'_femto'
    deploy_micro = folder+'deployment/'+str(i)+'_micro'
    deploy_pico = folder+'deployment/'+str(i)+'_pico'
    if os.path.isfile(deploy_femto) and os.path.isfile(deploy_micro) and os.path.isfile(deploy_pico):
        snr_femto = folder+'snr/'+str(i)+'_femto'
        snr_micro = folder+'snr/'+str(i)+'_micro'
        snr_pico = folder+'snr/'+str(i)+'_pico'
        if os.path.isfile(snr_femto) and os.path.isfile(snr_micro) and os.path.isfile(snr_pico):
            user_data = folder+'users'
            pathloss_data = folder+'pathloss'

            matriz_femto_snr = np.loadtxt(snr_femto, dtype=float)
            matriz_micro_snr = np.loadtxt(snr_micro, dtype=float)
            matriz_pico_snr = np.loadtxt(snr_pico, dtype=float)
            matriz_femto_deploy = np.loadtxt(deploy_femto, dtype=float)
            matriz_micro_deploy = np.loadtxt(deploy_micro, dtype=float)
            matriz_pico_deploy = np.loadtxt(deploy_pico, dtype=float)
            matriz_pathloss = np.loadtxt(pathloss_data, dtype=float)
            matriz_users = np.loadtxt(user_data, dtype=float)

            # Combina las matrices en una sola matriz de 8 dimensiones
            matriz_combinada = np.array([matriz_femto_snr, matriz_micro_snr, matriz_pico_snr,
                                         matriz_femto_deploy, matriz_micro_deploy, matriz_pico_deploy,
                                         matriz_pathloss, matriz_users])

            if estandarizacion:
              # Índices de las matrices que deseas estandarizar
              indices_estandarizar = [0, 1, 2, 6]

              # Calcula la media y la desviación estándar para las matrices seleccionadas
              medias = np.mean(matriz_combinada[indices_estandarizar], axis=(0, 1))
              desviaciones_estandar = np.std(matriz_combinada[indices_estandarizar], axis=(0, 1))

              # Estandariza las matrices seleccionadas
              for i in indices_estandarizar:
                  if desviaciones_estandar[i] != 0:
                      matriz_combinada[i] = (matriz_combinada[i] - medias) / desviaciones_estandar

            ruta_guardado_binario = folder_output+'IN/matriz_combinada'+str(i)+'.npy'
            np.save(ruta_guardado_binario, matriz_combinada)

            # Leer los datos desde el archivo CSV para generar los ficheros de salida con el que comparar las soluciones
            data = np.genfromtxt(folder+'usersInfo/'+str(i)+'.csv', delimiter=',', skip_header=1)

            # Obtener las coordenadas x, y y capacidad
            x = data[:, 1].astype(int)
            y = data[:, 2].astype(int)
            capacity = data[:, 4]

            # Obtener las dimensiones de la matriz
            max_x, max_y = np.max(x), np.max(y)

            # Crear una matriz de ceros con las dimensiones máximas
            matrix = np.zeros((100, 100))

            # Asignar los valores de capacidad en las posiciones correspondientes
            matrix[x, y] = capacity

            # Guardar la matriz en formato binario de NumPy (npy)
            ruta_guardado_binario = folder_output+'OUT/matriz_combinada'+str(i)+'.npy'
            np.save(ruta_guardado_binario, matrix)


## Se destina un conjunto de datos a TEST

In [56]:
import os
import random
import shutil

# Ruta del cual extraer las muestras para TEST'
folder_output = '/content/Matrices/'
lista_ficheros = os.listdir(folder_output+'IN/')
ruta_destino = folder_output+'TEST/IN'

random.shuffle(lista_ficheros)
random.shuffle(lista_ficheros)

num_archivos_validacion = int(len(lista_ficheros)*0.10)
archivos_validacion_completo = random.sample(lista_ficheros, num_archivos_validacion)

print(lista_ficheros[1],lista_ficheros[2])
print(len(archivos_validacion_completo))
print(archivos_validacion_completo[1],archivos_validacion_completo[2])

for archivo in archivos_validacion_completo:
    origenin = folder_output+'IN/'+archivo
    destinoin = folder_output+'TEST/IN/'+archivo
    shutil.move(origenin, destinoin)

    origenout = folder_output+'OUT/'+archivo
    destinoout = folder_output+'TEST/OUT/'+archivo
    shutil.move(origenout, destinoout)

matriz_combinada0.npy matriz_combinada218.npy
46
matriz_combinada208.npy matriz_combinada11.npy


# ENTRENAMIENTO DEL MODELO

### V6 versión modificada para permitir utilizar múltiples GPU's + Early Stopping

Dentro de train_v6.py, modificar las siguientes rutas:

* dir_img = Path('/opt/share/MERIDA/DATASET-TESTALL2/IN/')
* dir_mask = Path('/opt/share/MERIDA/DATASET-TESTALL2/OUT/')
* dir_checkpoint = Path('/opt/share/MERIDA/Code/Pytorch-UNet/checkpoints2/')
* early_stopping = EarlyStopping(patience=50, checkpoint_path='/opt/shareMERIDA/
Code/Pytorch-UNet/checkpoints/early_stop_checkpoint.pth')



In [ ]:
!python train_v6.py --amp --epochs 500

## Se copia el modelo ya entranado a la ruta requerida

Modelos subidos a https://mega.nz/folder/nBAVzJqZ#9_ODIDPU3kYwvX6t0r8Y_w

In [67]:
!cp /content/drive/MyDrive/MERIDAV2/checkpoint_NORMAL.pth /content/UNET5G/Code/Pytorch-UNet/MODEL.pth

# MEDIDAS EN BASE A LA ESTIMACIÓN REALIZADA

In [ ]:
import numpy as np

# Ruta al archivo .npy de salida, por ejemplo '/opt/share/MERIDA/out/matriz_combinada15.npy'
archivo_npy = '/content/Matrices/TEST/OUT/matriz_combinada1.npy'

matriz = np.load(archivo_npy)

# Obtener los índices de los elementos no cero
indices_no_cero = np.nonzero(matriz)

# Imprimir los valores no cero y sus índices
for i, j in zip(indices_no_cero[0], indices_no_cero[1]):
    valor = matriz[i, j]
    print(f"Valor no cero en la posición ({i}, {j}): {valor}")

# ESTIMACIÓN DE LA MATRIZ DE SALIDA EN BASE AL MODELO ENTRENADO

-i 'Matriz de dimensionalidad 8 destinada para TEST'
-o 'Matriz de dimensionalidad 1 con la estimación dada por el modelo'

In [60]:
cd /content/UNET5G/Code/Pytorch-UNet/

/content/UNET5G/Code/Pytorch-UNet


In [68]:
!python predictv3.py -i '/content/Matrices/TEST/IN/matriz_combinada1.npy' -o '/content/salidaexample1.npy'

INFO: Loading model MODEL.pth
INFO: Using device cuda
INFO: Model loaded!
INFO: Predicting values for matrix /content/Matrices/TEST/IN/matriz_combinada1.npy ...
(8, 100, 100)
INFO: Regression values saved to /content/salidaexample1.npy


#DIFERENCIA ENTRE LOS VALORES REALES Y LOS ESTIMADOS

In [ ]:
import numpy as np

# Cargar la matriz desde el archivo .npy con el GT, por ejemplo '/opt/share/MERIDA/DATASET-TESTALL/TEST/OUT/matriz_combinada43.npy'
matriz = np.load('/content/Matrices/TEST/OUT/matriz_combinada1.npy')

# Cargar la matriz desde el archivo .npy con las estimaciones, por ejemplo '/opt/share/MERIDA/SALIDA/salidabatch44.npy'
matriz_pred = np.load('/content/salidaexample1.npy')

# Obtener las posiciones donde los elementos no son cero
posiciones_no_cero = np.nonzero(matriz)
posiciones_cero = np.where(matriz==0)

valores_reales = []
valores_estimados = []

print('POSICIONES DISTINTAS DE CERO')
# Imprimir las posiciones y los valores correspondientes
for fila, columna in zip(*posiciones_no_cero):
    valor = matriz[fila, columna]
    valorpred = matriz_pred[0][0][fila,columna]
    print(f"Posición ({fila}, {columna}): Valor GT = {valor}, Valor PRED = {valorpred}")
    valores_reales.append(valor)
    valores_estimados.append(valorpred)

#print('POSICIONES DE CERO')
# Imprimir las posiciones y los valores correspondientes
#for fila, columna in zip(*posiciones_cero):
#    valor = matriz[fila, columna]
#    valorpred = matriz_pred[0][0][fila,columna]
#    print(f"Posición ({fila}, {columna}): Valor GT = {valor}, Valor PRED = {valorpred}")


#CÁLCULO DE MEDIDAS

In [70]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Convertir las listas a arrays de numpy
valores_reales = np.array(valores_reales)
valores_estimados = np.array(valores_estimados)

# Calcular las métricas
ecm = mean_squared_error(valores_reales, valores_estimados)
eam = mean_absolute_error(valores_reales, valores_estimados)
r_cuadrado = r2_score(valores_reales, valores_estimados)
emrc = np.sqrt(ecm)

# Imprimir los resultados
print("Error cuadrático medio (ECM):", ecm)
print("Error absoluto medio (EAM):", eam)
print("Coeficiente de determinación (R-cuadrado):", r_cuadrado)
print("Error medio de la raíz cuadrada (EMRC):", emrc)

Error cuadrático medio (ECM): 22.948255799960382
Error absoluto medio (EAM): 3.411395076761852
Coeficiente de determinación (R-cuadrado): 0.9668148631708507
Error medio de la raíz cuadrada (EMRC): 4.790433779936884
